In [3]:
# %%writefile vote_scraper.py
from pymongo import MongoClient
import pprint 
import pandas as pd 
import copy
from bs4 import BeautifulSoup
import requests
import datetime

from time import sleep
import warnings

client = MongoClient('mongodb://localhost:27017/')
db = client.bills
vote_records = db.vote_records

# the 101st Congress (1989 - 1990) starts on pg 1011 for pageSize=250
house_url_root = 'http://clerk.house.gov/evs'

date_range = list(range(1990, 2019))

for yr in date_range:
    site_url = '{}/{}/index.asp'.format(house_url_root, yr)
    req = requests.get(site_url)
    tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
    stat_code = req.status_code
    if stat_code != 200:
        print('_______________')
        print('_______________')
        print(site_url)
        print('Request Status Code: {}, {}'.format(stat_code, tstamp))
    else:
        print('No errors in requests!')
    

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [4]:
# test code with year 1990 for House votes
site_url_root = 'http://clerk.house.gov/evs'
yr = '1990'

site_url = '{}/{}/index.asp'.format(site_url_root, yr)
req = requests.get(site_url)
tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
stat_code = req.status_code
print('_______________')
print('_______________')
print(site_url)
print('Request Status Code: {}, {}'.format(stat_code, tstamp))

_______________
_______________
http://clerk.house.gov/evs/1990/index.asp
Request Status Code: 200, 12-10-2018 10:33:19


In [8]:
soup = BeautifulSoup(req.content, 'lxml')
# print(soup.prettify())

table = soup.find('table')
# print(table.prettify())

rows = table.find_all('tr')

In [25]:
# initial request of webpage will show the final table with the most recent roll call votes
# get the largest value of roll for iteration
final_roll = int(rows[1].find_all('a')[0].text.strip())

In [ ]:
vote_root_url = 'http://clerk.house.gov/evs/'

In [59]:
all_rows = []
empty_row = {
            "year": yr,
            "roll": None, 
            "date": None, 
            "issue": None,
            "question": None,
            "result": None,
            "description": None
            }

In [60]:
# skip the header when reading table
for row in rows[1:]:
    new_row = copy.copy(empty_row)
    columns = row.find_all('td')
    new_row['roll'] = columns[0].text.strip()
    new_row['issue'] = columns[2].text.strip()
    all_rows.append(new_row)

In [61]:
all_rows

[{'year': '1990', 'roll': '536', 'issue': 'S 280'},
 {'year': '1990', 'roll': '535', 'issue': 'H R 4793'},
 {'year': '1990', 'roll': '534', 'issue': 'S 3266'},
 {'year': '1990', 'roll': '533', 'issue': 'H J RES 687'},
 {'year': '1990', 'roll': '532', 'issue': 'H R 5769'},
 {'year': '1990', 'roll': '531', 'issue': 'H R 5114'},
 {'year': '1990', 'roll': '530', 'issue': 'S 358'},
 {'year': '1990', 'roll': '529', 'issue': 'H R 5399'},
 {'year': '1990', 'roll': '528', 'issue': 'H R 5835'},
 {'year': '1990', 'roll': '527', 'issue': 'H RES 537'},
 {'year': '1990', 'roll': '526', 'issue': 'H RES 531'},
 {'year': '1990', 'roll': '525', 'issue': 'S 1630'},
 {'year': '1990', 'roll': '524', 'issue': 'H RES 535'}]

In [ ]:
http://clerk.house.gov/evs/1990/roll524.xml
http://clerk.house.gov/evs/1990/roll001.xml

In [ ]:
# if req.status_code == 200:
#################
browser.get(site_url)
soup = BeautifulSoup(browser.page_source, 'lxml')
# print(soup.prettify())
div = soup.find('div', {'class':'row'})
print(div.prettify())

In [ ]:
# add page page html to mongo
collection_name.insert_one({'lxml': req.content})


# print result of load
with open('data/load_results.txt', 'a') as f:
    f.writelines('{}, {}, {}\n'.format(site_url, stat_code, tstamp))
f.close()


##################
# else: 
#     print('failed to get {}'.format(site_url))
#     # print result of load
#     with open('data/logs/load_results.txt', 'a') as f:
#         f.writelines('{}, {}, {}\n'.format(site_url, stat_code, tstamp))
#     f.close()